In [2]:
!pip install requests pymupdf
!pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [3]:
import requests
import fitz  # PyMuPDF
import re
from collections import Counter
from io import BytesIO
import PyPDF2

def download_pdf(url): # considering google share
    if 'drive.google.com' in url:
        file_id = url.split('/d/')[1].split('/view')[0]
        url = f"https://drive.google.com/uc?id={file_id}&export=download"
    response = requests.get(url)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception(f"Failed to download PDF: Status code {response.status_code}")

def extract_text_from_pdf(pdf_content): # get ccontent
    with fitz.open("pdf", pdf_content) as doc:
        text_pages = []
        for page in doc:
            text = page.get_text().replace("\n", " ")
            text_pages.append(text)
    return text_pages

def find_phone_numbers(text_pages):
    # Regular expression pattern for Hong Kong, London, and Lux phone numbers
    pattern1 = r'\(?\+?852\)?[-.\s]?\d{4}[-.\s]?\d{4}' # Hong Kong
    pattern2 = r'\(?\+?44\)?[-.\s]?\d{2,3}[-.\s]?\d{4}[-.\s]?\d{4}' # London
    pattern3 = r'\(?\+?352\)?[-.\s]?\d{3}[-.\s]?\d{3}[-.\s]?\d{3}' # Lux
    pattern4 = r'\(?\+?020\)?[-.\s]?\d{4}[-.\s]?\d{4}' # London
    pattern5 = r'\(?\+?65\)?[-.\s]?\d{4}[-.\s]?\d{4}' # Sing
    pattern6 = r'\(?\+1\)?[-.\s]?\d{3}[-.\s]?\d{3}[-.\s]?\d{4}' # US
    pattern7 = r'\(?\+?27\)?[-.\s]?\d{2}[-.\s]?\d{3}[-.\s]?\d{4}' # South Afr
    pattern8 = r'\(?\+?41\)?[-.\s]?\d{2}[-.\s]?\d{3}[-.\s]?\d{2}[-.\s]?\d{2}' # Swi

    phone_numbers = []
    phone_page_numbers = []
    for i, text in enumerate(text_pages):
        text = text.replace(" ", "")
        phone_numbers_this_page = (
            re.findall(pattern1, text, re.DOTALL) +
            re.findall(pattern2, text, re.DOTALL) +
            re.findall(pattern3, text, re.DOTALL) +
            re.findall(pattern4, text, re.DOTALL) +
            re.findall(pattern5, text, re.DOTALL) +
            re.findall(pattern6, text, re.DOTALL) +
            re.findall(pattern7, text, re.DOTALL) +
            re.findall(pattern8, text, re.DOTALL)
        )

        if phone_numbers_this_page:
            phone_numbers.extend(phone_numbers_this_page)
            phone_page_numbers.append(i + 1)  # page numbers are 1-indexed

    phone_numbers = [
        ph.replace(".", "")
          .replace("-", "")
          .replace("(","")
          .replace(")", "")
          .replace("/s", "")
        for ph in phone_numbers
    ]

    phone_counter = Counter(phone_numbers)
    ph, _ = phone_counter.most_common(1)[0] if phone_counter else (None, None)
    print(ph)
    return ph, phone_page_numbers

def find_emails(text_pages):
    # Regular expression pattern for emails
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}\b'
    emails = []
    email_page_numbers = []
    for i, text in enumerate(text_pages):
        # Convert all emails to lowercase
        emails_this_page = re.findall(pattern, text, re.DOTALL)

        if emails_this_page:
            emails.extend(emails_this_page)
            email_page_numbers.append(i + 1)  # page numbers are 1-indexed

    emails = [email.lower() for email in emails]
    email_counter = Counter(emails)
    e, _ = email_counter.most_common(1)[0] if email_counter else (None, None)

    return e, email_page_numbers

def main(url):
    phone_numbers=None
    emails = None
    email_page_numbers = None
    phone_page_numbers = None
    try:
        pdf_content = download_pdf(url)
        text_pages = extract_text_from_pdf(pdf_content)

        phone_numbers, phone_page_numbers = find_phone_numbers(text_pages)
        emails, email_page_numbers = find_emails(text_pages)

        #print("Found phone numbers:", phone_numbers)
        #print("Found emails:", emails)

    except Exception as e:
        print("An error occurred:", str(e))

    return (phone_page_numbers, phone_numbers, email_page_numbers, emails)

#main("https://drive.google.com/file/d/16X-T8FZvJBqHRofruor0odieOOPotJKX/view?usp=sharing")

+85236080306


([7], '+85236080306', [], None)

In [ ]:
import pandas as pd

df = pd.read_csv('input.csv')
df_a = pd.DataFrame(df[['Fund Name','ISIN']])

prospectus = df['Prospectus'].tolist()

page_number_of_phones = []
page_number_of_emails = []
phones=[]
emails=[]

for p in prospectus:
    #print(main(p))
    result = main(p)
    page_number_of_phones.append(result[0])
    phones.append(result[1])
    page_number_of_emails.append(result[2])
    emails.append(result[3])

df_a["phone"] = phones
df_a["email"] = emails
df_a["phone_page"]=page_number_of_phones
df_a["email_page"]=page_number_of_emails
df_a.to_csv('output.csv', index=False)
#print(phones,emails)


+85236507700
+85239032688
+85239032688
+85239032688
+85239032688
+85239032688
85226292629
85228777733
85228777733
85228777733
85228777733
85226292629
85226292629
85226292629
+85239032688
None
+85239032688
+85239032688
+85239032688
+85239032688
+85236269700
+85236269700
85221178383
85221178383
85221178383
None
None
None
None
None
None
None
+85229187888
None
None
None
None
None
None
None
None
None
+85236507700
None
None
None
85225214231
None
None
+85239565816
+85239565816
+352474066425
+85239032688
+352474066425
None
+85239032688
+352474066425
+352474066425
+352474066425
85228777733
None
None
None
None
85226292629
None
None
85225330088
85225330088
An error occurred: argument of type 'float' is not iterable
None
None
85228411411
None
+85229187888
+85228050111
85226292629
+85239032688
+85239032688
+85239032688
+85236507700
85226292629
85228411411
+85236507700
+85236648868
+85231918282
+85236648868
None
None
+85236507700
+85231918282
None
+85228405388
+85228405388
+85228405388
85228777733
+